In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px
import eli5
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMRegressor
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import BayesianRidge, LinearRegression, ElasticNet  # 批量导入要实现的回归算法
from sklearn.svm import SVR  # SVM中的回归算法
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor  # 集成算法
from sklearn.model_selection import cross_val_score  # 交叉检验
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score  # 批量导入指标算法

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data2015 = pd.read_csv("../input/world-happiness/2015.csv")
data2015['year'] = 2015
data2015.rename(columns={'Dystopia Residual': 'Dystopia_Residual', 
                         'Economy (GDP per Capita)': 'Economy_GDP_per_Capita', 
                         'Happiness Rank': 'Happiness_Rank', 
                         'Happiness Score': 'Happiness_Score', 
                         'Health (Life Expectancy)': 'Health(Life_Expectancy)', 
                         'Lower Confidence Interval': 'Lower_Confidence_Interval', 
                         'Standard Error': 'Standard_Error', 
                         'Trust (Government Corruption)': 'Trust(Government_Corruption)'}, 
                inplace=True)
del data2015['Standard_Error']
# del data2015['Dystopia_Residual']
data2015.info()

In [ ]:
data2016 = pd.read_csv("../input/world-happiness/2016.csv")
data2016['year'] = 2016
data2016.rename(columns={'Dystopia Residual': 'Dystopia_Residual', 
                         'Economy (GDP per Capita)': 'Economy_GDP_per_Capita', 
                         'Happiness Rank': 'Happiness_Rank', 
                         'Happiness Score': 'Happiness_Score', 
                         'Health (Life Expectancy)': 'Health(Life_Expectancy)', 
                         'Lower Confidence Interval': 'Lower_Confidence_Interval',
                         'Upper Confidence Interval': 'Upper_Confidence_Interval',
                         'Standard Error': 'Standard_Error', 
                         'Trust (Government Corruption)': 'Trust(Government_Corruption)'}, 
                inplace=True)
del data2016['Lower_Confidence_Interval']
del data2016['Upper_Confidence_Interval']
# del data2016['Dystopia_Residual']
# d2016_nodr = pd.DataFrame(data2016.iloc[:, [i for i in range(0,10)]])
data2016.info()

In [ ]:
data2017 = pd.read_csv("../input/world-happiness/2017.csv")
data2017['year'] = 2017
data2017.rename(columns={'Dystopia.Residual': 'Dystopia_Residual', 
                         'Economy..GDP.per.Capita.': 'Economy_GDP_per_Capita', 
                         'Happiness.Rank': 'Happiness_Rank', 
                         'Happiness.Score': 'Happiness_Score', 
                         'Health..Life.Expectancy.': 'Health(Life_Expectancy)', 
                         'Whisker.low': 'Lower_Confidence_Interval', 
                         'Whisker.high': 'Upper_Confidence_Interval', 
                         'Trust..Government.Corruption.': 'Trust(Government_Corruption)'}, 
                inplace=True)
del data2017['Lower_Confidence_Interval']
del data2017['Upper_Confidence_Interval']
# del data2017['Dystopia_Residual']
data2017.info()

In [ ]:
data2018 = pd.read_csv("../input/world-happiness/2018.csv")
data2018['year'] = 2018
data2018.rename(columns={'Dystopia.Residual': 'Dystopia_Residual', 
                         'GDP per capita': 'Economy_GDP_per_Capita', 
                         'Overall rank': 'Happiness_Rank', 
                         'Score': 'Happiness_Score', 
                         'Healthy life expectancy': 'Health(Life_Expectancy)', 
                         'Whisker.low': 'Lower_Confidence_Interval', 
                         'Whisker.high': 'Upper_Confidence_Interval',
                         'Standard Error': 'Standard_Error', 
                         'Perceptions of corruption': 'Trust(Government_Corruption)',
                         'Freedom to make life choices':'Freedom',
                         'Social support':'Family',
                         'Country or region':'Country'
                        }, 
                inplace=True)
data2018.info()

In [ ]:
data2019 = pd.read_csv("../input/world-happiness/2019.csv")
data2019['year'] = 2019
data2019.rename(columns={'Dystopia.Residual': 'Dystopia_Residual', 
                         'GDP per capita': 'Economy_GDP_per_Capita', 
                         'Overall rank': 'Happiness_Rank', 
                         'Score': 'Happiness_Score', 
                         'Healthy life expectancy': 'Health(Life_Expectancy)', 
                         'Whisker.low': 'Lower_Confidence_Interval', 
                         'Whisker.high': 'Upper_Confidence_Interval',
                         'Standard Error': 'Standard_Error', 
                         'Perceptions of corruption': 'Trust(Government_Corruption)',
                         'Freedom to make life choices':'Freedom',
                         'Social support':'Family',
                         'Country or region':'Country'
                        }, 
                inplace=True)
data2019.info()

In [ ]:
full_data = pd.concat([data2015, data2016, data2017, data2018, data2019])
full_data = full_data.drop(['Region'], axis = 1)

# full_data = full_data.dropna()
# full_data.info()
full_data.fillna(full_data.mean(), inplace = True)
# full_data.info()

In [ ]:
trace = dict(type = 'choropleth', 
           locations = full_data['Country'],
           locationmode = 'country names',
           z = full_data['Happiness_Score'], 
           text = full_data['Country'],
           colorbar = {'title':'Happiness'})
layout = dict(title = 'Global Happiness', 
             geo = dict(showframe = False))
choromap3 = go.Figure(data = [trace], layout=layout)
iplot(choromap3)

幸福指数的全球分布情况：
从地图可以看出北欧、澳洲以及北美洲是幸福指数较高的地区，而非洲和西亚是幸福指数较低的地区，其他地区如南美洲和东亚则处于中间位置。
这与我们从新闻时政消息当中获得的常识吻合：北欧和北美洲发达国家较多，环境优美，人们生活压力较小。而非洲和西亚则是落后国家密集的地方，人们尚未摆脱疾病、战争和饥饿的困扰，因此幸福指数较低。

In [ ]:
f,ax = plt.subplots(figsize = (11, 11))
sns.heatmap(full_data.corr(),annot = True, linewidth = .5,fmt = '.1f',ax = ax)
plt.show()

幸福指数与哪些因素有关呢？
画出变量之间的热力关系图来探究，颜色越深表示越负相关，颜色越浅表示越正相关。
1. 排名与指数呈负相关关系，指数越高，排名越靠前，这是自然的
2. 人均GDP、家庭、选择自由度、寿命健康均与指数有较高的正相关系数，这与常识相符
3. 对国家的信任感、反乌托邦残余、宽容程度与指数也有些许正相关关系、
下面通过具体作图进行线性拟合。

In [ ]:
fig = px.scatter(full_data, 
                 x = "Economy_GDP_per_Capita", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'GDP per capita and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
                 x = "Family", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'Family and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
                 x = "Health(Life_Expectancy)", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'Healthy Life Expecancy and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
                 x = "Freedom", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'Freedom and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
                 x = "Generosity", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'Generosity and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
                 x = "Trust(Government_Corruption)", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'Trust(Government_Corruption) and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
                 x = "Dystopia_Residual", 
                 y = "Happiness_Score",
                 facet_row = "year",
                 color="year",
                 trendline= "ols")
fig.update(layout_coloraxis_showscale = False)
fig.update_traces(textposition = 'top center')
fig.update_layout(
    height = 800,
    title_text = 'Dystopia_Residual and Happiness Score'
)
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Economy_GDP_per_Capita", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Family", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Health(Life_Expectancy)", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Freedom", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Generosity", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Trust(Government_Corruption)", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
fig = px.scatter(full_data, 
           x = "Dystopia_Residual", 
           y = "Happiness_Score", 
           animation_frame = "year",
           animation_group="Country",
           size = "Happiness_Rank", 
           color="Country", 
           hover_name="Country",
           trendline= "ols")
fig.show()

In [ ]:
y = full_data.loc[:, 'Happiness_Score']
X = full_data.loc[:, "Economy_GDP_per_Capita":"year"]

# 训练回归模型
n_folds = 6  # 设置交叉检验的次数
model_br = BayesianRidge()  # 建立贝叶斯岭回归模型对象
model_lr = LinearRegression()  # 建立普通线性回归模型对象
model_etc = ElasticNet(l1_ratio = 0, max_iter = 10000000, alpha = 3)  # 建立弹性网络回归模型对象
model_svr = SVR(C = 1000000)  # 建立支持向量机回归模型对象
model_gbr = GradientBoostingRegressor()  # 建立梯度增强回归模型对象
model_names = ['BayesianRidge', 'LinearRegression', 'ElasticNet', 'SVR', 'GBR']  # 不同模型的名称列表
model_dic = [model_br, model_lr, model_etc, model_svr, model_gbr]  # 不同回归模型对象的集合
cv_score_list = []  # 交叉检验结果列表
pre_y_list = []  # 各个回归模型预测的y值列表

for model in model_dic:  # 读出每个回归模型对象
    scores = cross_val_score(model, X, y, cv=n_folds)  # 将每个回归模型导入交叉检验模型中做训练检验
    cv_score_list.append(scores)  # 将交叉检验结果存入结果列表
    pre_y_list.append(model.fit(X, y).predict(X))  # 将回归训练中得到的预测y存入列表

# 模型效果指标评估
n_samples, n_features = X.shape  # 总样本量,总特征数
model_metrics_name = [explained_variance_score, mean_absolute_error, mean_squared_error, r2_score]  # 回归评估指标对象集
model_metrics_list = []  # 回归评估指标列表
for i in range(5):  # 循环每个模型索引
    tmp_list = []  # 每个内循环的临时结果列表
    for m in model_metrics_name:  # 循环每个指标对象
        tmp_score = m(y, pre_y_list[i])  # 计算每个回归指标结果
        tmp_list.append(tmp_score)  # 将结果存入每个内循环的临时结果列表
    model_metrics_list.append(tmp_list)  # 将结果存入回归评估指标列表
df1 = pd.DataFrame(cv_score_list, index=model_names)  # 建立交叉检验的数据框
df2 = pd.DataFrame(model_metrics_list, index=model_names, columns=['ev', 'mae', 'mse', 'r2'])  # 建立回归指标的数据框

print ('samples: %d \t features: %d' % (n_samples, n_features))  # 打印输出样本量和特征数量
print (70 * '-')  # 打印分隔线
print ('cross validation result:')  # 打印输出标题
print (df1)  # 打印输出交叉检验的数据框
print (70 * '-')  # 打印分隔线
print ('regression metrics:')  # 打印输出标题
print (df2)  # 打印输出回归指标的数据框
print (70 * '-')  # 打印分隔线
print ('short name \t full name')  # 打印输出缩写和全名标题
print ('ev \t explained_variance')
print ('mae \t mean_absolute_error')
print ('mse \t mean_squared_error')
print ('r2 \t r2')
print (70 * '-')  # 打印分隔线


Dystopia_Residual后两年存在大量缺失值,用平均值填充的训练效果好于用0填充的效果。

GBR表现最好，贝叶斯岭回归和线性回归非常接近，SVR排第三名，最后弹性网络效果很差。

In [ ]:
# 模型效果可视化
plt.figure(figsize = (10, 10))  # 创建画布
plt.plot(np.arange(X.shape[0]), y, color='k', label='true y')  # 画出原始值的曲线
color_list = ['r', 'b', 'g', 'y', 'c']  # 颜色列表
linestyle_list = ['-', '.', 'o', 'v', '*']  # 样式列表
for i, pre_y in enumerate(pre_y_list):  # 读出通过回归模型预测得到的索引及结果
    plt.plot(np.arange(X.shape[0]), pre_y_list[i], color_list[i], label=model_names[i])  # 画出每条预测结果线
plt.title('regression result comparison')  # 标题
plt.legend(loc='upper right')  # 图例位置
plt.ylabel('real and predicted value')  # y轴标题
plt.show()  # 展示图像

可以看到前三年的数据拟合程度好于后两年的拟合程度，原因就在于反乌托邦残余这一变量。

In [ ]:
lgbm = LGBMRegressor(n_estimators = 5000)
indData = full_data.loc[:,"Economy_GDP_per_Capita":"year"]
depData = full_data["Happiness_Score"]
lgbm.fit(indData, depData)

columns = indData.columns.to_list()
perm = PermutationImportance(lgbm, random_state=10).fit(indData, depData)
eli5.show_weights(perm, feature_names = columns)

用LGBM查看各个特征的重要性，果然反乌托邦残余很重要，排在第二位，仅次于人均GDP。

家庭和健康这两个指标的重要程度类似

自由度和对政府的信任程度这两个指标的权重都很小，发挥作用不大。